<a href="https://colab.research.google.com/github/Ragul-MFAI/True_fake-prediction/blob/main/True_and_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TRUE AND FAKE NEWS PRDEICTION**

--> In this project we intend to predict the true and fake news that has been published in the journals and newspapers

**DataSet Description**

---> The Project dataset have been splitted into two sets the **TRAIN** and the **TEST** datasets.

---> The whole data has about of values where the TRAINING SET contain 20800 records with 5 attributes and the TESTING SET contain 5200 records with 4 attributes.



*   id - Its the unique number or value given to each and every News
*   title - Tells us the title or heading of the particular News
*   author -  The person who published the News
*   text - The elaborative explanation of the Event,Happenings etc in the city or Country
*   label - Its the labelling to true and fake News 0 for fake News and 1 for True News.






## HYPOTHESIS GENERATION

--> Fake News tend to express more sentiment as compared to the True News which are more sublte and balanced in tone.

## IMPORTING LIBRARIES

---> Importing the essential libraries into the python environment like Numpy,Pandas,Matplotlib,Seaborn which are used in the data loading,data cleaning and data preprocessing processes.




In [ ]:
# For file handling and usage
import numpy as np
import pandas as pd

# For Sentimental Analysis
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


# For building the machine learning models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# For evaluating the models using the metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection  import cross_val_score as CVS
from sklearn.metrics import classification_report

# For Saving the model
import pickle

# To avoid Warings
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

## LOADING THE DATA

* The two TRAIN and the TEST datasets have been uploaded separately as train and test into the environment.

* Here The datasets have been uploaded directly and the colab is mounted with the the drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Mounting my google drive to the Google Colab Notebook where the dataset have been uploaded already.
# So that we need not upload the dataset every time the runtime is disconnected.

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/DataSets/True and Fake News Prediction/train.csv")
# Reading the given  CSV dataset using the pandas libraries.

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/DataSets/True and Fake News Prediction/test.csv")

## LOOKING INTO THE DATA

--> Here we analyse the data like how many records in there, whether they have any null values or not and we try to tackle them acoordingly.



In [ ]:
train.shape
# This function hepls us identify the number of rows and columns present in the data.

(20800, 5)

In [ ]:
test.shape
# The same is been done with the test  set as well.

(5200, 4)

In [ ]:
train.info()
# The above function returns the detailed information of the uploaded data alongwith their count as well as their datatype.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


* From the above information it clear that the **id** and **label** column from the data is the only numerical datatype where others are just float.

* And also there are missing values that is present in both the train and test sets which need to be sortd out.

In [ ]:
train.isnull().sum()
# This Function gives the count of all the null values present in the data

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [ ]:
train.head()
# The head function shows the first five data of the entire dataset.

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


## DATA PREPROCESSING

---> The data that we loked into has number of missing values which should be processed with the following data preprocesing steps.

* 1. Removing the null values
* 2. Tokenisation
* 3. Lowercasing the text
* 4. Removal of Punctuations
* 5. Removal of
* 6. Stemming


**1.Removing Null values**

In [ ]:
train = train.fillna('')
# Here we intend to fill all the null values in the training set with the empty space.

In [ ]:
test = test.fillna('')

In [ ]:
train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
test.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

The **zeros** represents that the null values have been removed.

In [ ]:
train['subject'] = train['author'] + '' + train['title']
# Merging the two columns author and the text with the empty space for the upcoming data preprocessing.

In [ ]:
train.head()
#train = train.drop('subject',axis = 1)

,id,title,author,text,label,subject
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...


**2.Tokenisation**


In [ ]:
#from nltk.tokenize import word_tokenize

# Select 'title' and 'text' columns and apply word_tokenize to each row
#train['tokens'] = train[['title', 'text']].apply(lambda row: word_tokenize(row['title']) + word_tokenize(row['text']), axis=1)

In [ ]:
!pip install nltk # Make sure NLTK is installed
import nltk
nltk.download('punkt') # Download the 'punkt' resource
from nltk.tokenize import word_tokenize
train['tokens'] = train['subject'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train.head()

,id,title,author,text,label,subject,tokens
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...,"[Darrell, LucusHouse, Dem, Aide, :, We, Didn, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...","[Daniel, J., FlynnFLYNN, :, Hillary, Clinton, ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...,"[Consortiumnews.comWhy, the, Truth, Might, Get..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...,"[Jessica, Purkiss15, Civilians, Killed, In, Si..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...,"[Howard, PortnoyIranian, woman, jailed, for, f..."


**3.LOWERCASING THE TEXT**

In [ ]:
train['subject'] = train['subject'].str.lower()
# We convert every uppercase letter into lowercase ones using .str.lower() function

In [ ]:
train.head()

,id,title,author,text,label,subject,tokens
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrell lucushouse dem aide: we didn’t even se...,"[Darrell, LucusHouse, Dem, Aide, :, We, Didn, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"daniel j. flynnflynn: hillary clinton, big wom...","[Daniel, J., FlynnFLYNN, :, Hillary, Clinton, ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnews.comwhy the truth might get you ...,"[Consortiumnews.comWhy, the, Truth, Might, Get..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss15 civilians killed in single u...,"[Jessica, Purkiss15, Civilians, Killed, In, Si..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoyiranian woman jailed for fiction...,"[Howard, PortnoyIranian, woman, jailed, for, f..."


**4.REMOVAL OF PUNCTUATIONS**

In [ ]:
train['subject'] = train['subject'].str.replace('[^\w\s]', '')
# The str.replace function replaces all the punctuations buy filling up the space instead.

In [ ]:
train.head()

,id,title,author,text,label,subject,tokens
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrell lucushouse dem aide: we didn’t even se...,"[Darrell, LucusHouse, Dem, Aide, :, We, Didn, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"daniel j. flynnflynn: hillary clinton, big wom...","[Daniel, J., FlynnFLYNN, :, Hillary, Clinton, ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnews.comwhy the truth might get you ...,"[Consortiumnews.comWhy, the, Truth, Might, Get..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss15 civilians killed in single u...,"[Jessica, Purkiss15, Civilians, Killed, In, Si..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoyiranian woman jailed for fiction...,"[Howard, PortnoyIranian, woman, jailed, for, f..."


**5.REMOVING STOPWORDS**

In [ ]:
nltk.download('stopwords')
# Downloading the stopwords that are present in English Langugae

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
SW = set(stopwords.words('english'))
print(SW)
# Showcasing the stopwords in English

{'as', 'i', 'of', 'few', 'other', 'again', 'only', 'no', "shan't", "don't", 'up', 'with', 'both', "that'll", 'shouldn', 'between', "you'd", 'and', 'y', "couldn't", 'its', 'haven', 'why', 'so', 'this', 'further', 'in', "hasn't", 'will', 'yourself', "needn't", "hadn't", 'should', 'or', 'what', 'each', "doesn't", 'didn', 'any', 'now', 'there', 'did', 'o', 'we', 'was', 'had', 'not', 'himself', 'wouldn', 'whom', 'off', 'ain', 'out', 'over', 'his', 'yours', 'my', 'nor', 'who', 'under', "mightn't", "wasn't", 'me', 'while', 're', 'before', 'being', 'd', 'ours', 'is', 'isn', 'hasn', 'too', 'our', 'shan', 'have', 'mustn', 'has', 'been', "should've", 'then', 'their', 'at', "you're", 'down', 'a', 'how', 've', 'when', 'don', 'hadn', 'because', 'wasn', 'during', 'on', 'myself', 'them', "shouldn't", 'an', 'after', "you've", 'through', 'theirs', 'such', 'am', 'which', 'm', 'these', 'themselves', 'more', 'couldn', 'do', 'yourselves', 'until', "you'll", 'doing', 'than', 'doesn', 'above', 'needn', "it's"

In [ ]:
train['subject'] = train['subject'].apply(lambda x: ' '.join([word for word in x.split() if word not in SW]))
# Here we try to remove the stopwords by adding only the words which are not in the SW stopword set.

In [ ]:
train.tail()

,id,title,author,text,label,subject,tokens
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerome hudsonrapper t.i.: trump ’poster child ...,"[Jerome, HudsonRapper, T.I, ., :, Trump, a, ’,..."
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"benjamin hoffmann.f.l. playoffs: schedule, mat...","[Benjamin, HoffmanN.F.L, ., Playoffs, :, Sched..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j. de la merced rachel abramsmacy’s sa...,"[Michael, J., de, la, Merced, and, Rachel, Abr..."
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"alex ansarynato, russia hold parallel exercise...","[Alex, AnsaryNATO, ,, Russia, To, Hold, Parall..."
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1,david swansonwhat keeps f-35 alive,"[David, SwansonWhat, Keeps, the, F-35, Alive]"


**6.STEMMING OR LEMMATIZATION**

In [ ]:
stemmer = PorterStemmer()
train['subject'] = train['subject'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
# Stemming is the simple process of reducing the words to their root forms

In [ ]:
train.head()

,id,title,author,text,label,subject,tokens
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucushous dem aide: didn’t even see com...,"[Darrell, LucusHouse, Dem, Aide, :, We, Didn, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"daniel j. flynnflynn: hillari clinton, big wom...","[Daniel, J., FlynnFLYNN, :, Hillary, Clinton, ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnews.comwhi truth might get fire,"[Consortiumnews.comWhy, the, Truth, Might, Get..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss15 civilian kill singl us airst...,"[Jessica, Purkiss15, Civilians, Killed, In, Si..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoyiranian woman jail fiction unpub...,"[Howard, PortnoyIranian, woman, jailed, for, f..."


## BUILDING UP THE MODEL

In [ ]:
X = train['subject']
Y = train['label']
# Splitting  up the features and the target Attributes separately before we vectorize everything


In [ ]:
X.head()
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [ ]:
X.shape

(20800,)

In [ ]:
Y.shape

(20800,)

**FINDING TF-IDF**

--> TF is the **TERM FREQUENCY** which is used to find how frequently a word ocurs in a given document.

--> IDF is the **INVERSE DOCUMENT FREQUENCY** how important a term is within the entire corpus. It reduces the weight of terms that appear very frequently across many documents and increases the weight of terms that appear less frequently.

--> TF-IDF value for a term  is the product of its TF and IDF values.

--> Its mainly used in Feature Extraction,Information Retrival and in Text Mining.

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)
# We intend to convert the text values into Numerical ones using the vectorizer.

In [ ]:
print(X)

  (0, 30011)	0.2702317405535354
  (0, 25953)	0.23633175774140575
  (0, 17287)	0.39616172821890755
  (0, 16772)	0.2687730240968852
  (0, 14864)	0.2267902704938032
  (0, 9503)	0.2131010345817379
  (0, 7770)	0.2725285388588018
  (0, 7469)	0.26535031144709154
  (0, 7127)	0.3269077189450684
  (0, 5991)	0.22776883917952795
  (0, 5117)	0.3339547020931297
  (0, 1090)	0.36074580999296135
  (1, 31859)	0.36688013213305515
  (1, 13172)	0.2508638367430399
  (1, 10395)	0.5226747357728401
  (1, 7068)	0.31638169487081524
  (1, 5725)	0.23248708292981804
  (1, 4720)	0.464055696418436
  (1, 4070)	0.18539331285820687
  (1, 3346)	0.35413476846519715
  (2, 29899)	0.36635739112785753
  (2, 18743)	0.4315491786494108
  (2, 11172)	0.3064339824029749
  (2, 10169)	0.3472323577842834
  (2, 6291)	0.404084133229652
  :	:
  (20797, 28830)	0.3162590151970751
  (20797, 25250)	0.2481398859255049
  (20797, 23768)	0.2719892730679363
  (20797, 23448)	0.24910787789148428
  (20797, 20003)	0.08061747005948777
  (20797, 18695)

**SPLITTING UP THE TRAINING AND TESTING TESTS**


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,stratify = Y,random_state = 42)
# Splitting up the data for traning the model.

In [ ]:
# Define the linear regression model
lr = LinearRegression()

# Train the model on the training data
lr.fit(X_train, Y_train)

# Precting the model using the test set
Y_predict = lr.predict(X_train)

# Evaluating the training set with the metric values

print("R-Square value on the training set: = ", lr.score(X_train,Y_train))
# Calculate the mean squared error
mse_train = mean_squared_error(Y_train, Y_predict)
print("Mean Squared Error in Training Set : = ",mse_train)

print("----------------------------------------------------------------------------------------------")

# Use cross-validation to get the R-squared score
scores = CVS(lr, X_train, Y_train, cv=5)
print('CV R-squared scores:', scores,"\n")
print('Average CV R-squared score:', np.mean(scores),"\n")

R-Square value on the training set: =  0.9999999831353436
Mean Squared Error in Training Set : =  4.2161580326763525e-09
----------------------------------------------------------------------------------------------
CV R-squared scores: [0.76935247 0.77872548 0.78168238 0.78233157 0.78930021] 

Average CV R-squared score: 0.7802784229725898 



In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

In [ ]:
X_tr_predict =model.predict(X_train)

accuracy =accuracy_score(Y_train,X_tr_predict)
precision = precision_score(Y_train, X_tr_predict)
recall = recall_score(Y_train, X_tr_predict)
f1 = f1_score(Y_train, X_tr_predict)

print(' Training Accuracy:', accuracy)
print(' Training Precision:', precision)
print(' Training Recall:', recall)
print('Training F1 Score:', f1)



 Training Accuracy: 0.971875
 Training Precision: 0.9519429753966429
 Training Recall: 0.9939975990396158
Training F1 Score: 0.9725158562367864


In [ ]:
X_tt_predict =model.predict(X_test)

accuracy =accuracy_score(Y_test,X_tt_predict)
precision = precision_score(Y_test,X_tt_predict)
recall = recall_score(Y_test,X_tt_predict)
f1 = f1_score(Y_test,X_tt_predict)

print(' Testing Accuracy:', accuracy)
print(' Testing  Precision:', precision)
print(' Testing  Recall:', recall)
print(' Testing  F1 Score:', f1)



 Testing Accuracy: 0.9504807692307692
 Testing  Precision: 0.9180400890868596
 Testing  Recall: 0.9894383101296207
 Testing  F1 Score: 0.9524029574861367


**Analysing using the Confusion Matrix**

## SAVING THE MODEL

---> We save the model so that we need not retrain the model again for the unseen data once again.

---> The saved models is used for the depolyment purposes were it can be used in diffrernt environments.

---> It avoids inconsistencies that arises by retraining the model again and again.

In [ ]:
tfp = "True_Fake_News_Prediction.sav"
pickle.dump(model, open(tfp,'wb'))

## SUMMARY

--- >  In this project We intend to find the true news and fake news using machine learning.

---> There were missing values in the dataset which was sorted out in the Data Preprocessing step

---> In the data preprocessing we also did Removing the null values, Tokenisation ,Lowercasing the text, Removal of Punctuations, Removal of Stopwords, Stemming.

---> Followed by this we build the machine leraning model using the linear regression and as well as logistic regression.

---> The linear Regression model worked pretty much well in the training set with the R Square value of 0.99 but when we cross validate the model's CV R Square value dropped to 0.77

---> This led to the use of the logistic regression where the mdel performed pretty much good with the acuuracy of 0.97 on the training set an 0.95 in the testing.

---> The above result clearly teel sthat the Logistic regressor worked well with high performance on both the dataset

---> Therefore the model generalises well and is effective for the classifier dataset.
